In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding,Lambda,Dense
import keras.backend as K
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import gensim

In [ ]:
from tensorflow.python.keras import utils

In [ ]:
data=open('CBOW.txt','r')
c_data=[text for text in data if text.count(' ')>=2]
c_data

In [ ]:
vectorizer=Tokenizer()
vectorizer.fit_on_texts(c_data)
c_data=vectorizer.texts_to_sequences(c_data)

In [ ]:
total_vocab=sum(len(s) for s in c_data)
word_count=len(vectorizer.word_index)+1
window_size=2
total_vocab
word_count

In [ ]:
def cbow_model(data,window_size,total_vocab):
  total_length=window_size*2
  for text in data:
    text_len=len(text)
    for idx,word in enumerate(text):
      context=[]
      target=[]
      begin=idx-window_size
      end=idx+window_size+1
      context.append([text[i] for i in range(begin,end) if 0<=i<text_len and i!=idx])
      target.append(word)
      contextual=sequence.pad_sequences(context,total_length)
      final_target=utils.to_categorical(target,total_vocab)
      yield(contextual,final_target)

In [ ]:
model=Sequential()
model.add(Embedding(input_dim=total_vocab,output_dim=100,input_length=window_size*2))
model.add(Lambda(lambda x: K.mean(x,axis=1),output_shape=(100,)))
model.add(Dense(total_vocab,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crooentropy',optimizer='adam')

In [ ]:
for i in range(10):
  cost=0
  for x,y in cbow_model(data,window_size,total_vocab):
    cost+=model.train_on_batch(x,y)

In [ ]:
dimensions=100
vect_file=open('vector.txt','w')
vect_file.write('{}  {}\n'.format(total_vocab,dimensions))

In [ ]:
weights=model.get_weights()[0]
for text,i in vectorizer.word_index.items():
  final_vec=' '.join(map(str,list(weights[i, :])))
  vect_file.write('{} {}\n'.format(text,final_vec))
vect_file.close()

In [ ]:
cbow_model=gensim.models.KeyedVectors.load_word2vec_format('vector.txt',binary=False,limit=100)

In [ ]:
cbow_model.most_similar(positive=['virus'])